# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [1]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

## Perform stepwise selection

The function for stepwise selection is copied below. Use this function provided on your preprocessed Boston Housing data.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [3]:
preprocessed

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.259100,-0.465447,0.416538,0.121392,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1456,0.725171,1.980456,1.106213,0.577822,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1457,-0.002324,0.228260,1.469438,1.174306,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1458,0.136814,-0.077546,-0.854179,-0.399519,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
predictors = preprocessed.drop('SalePrice_log', axis =1)

In [ ]:
# Your code here
result = stepwise_selection(predictors, preprocessed['SalePrice_log'], verbose=True)
print('resulting features:')
print(result)

/opt/conda/envs/learn-env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Add  GrLivArea_log                  with p-value 1.59847e-243
Add  KitchenQual_TA                 with p-value 1.56401e-67
Add  1stFlrSF_log                   with p-value 7.00069e-48
Add  KitchenQual_Fa                 with p-value 1.70471e-37
Add  Neighborhood_OldTown           with p-value 3.20105e-23
Add  KitchenQual_Gd                 with p-value 4.12635e-21
Add  Neighborhood_Edwards           with p-value 9.05184e-17
Add  Neighborhood_IDOTRR            with p-value 1.10068e-18
Add  LotArea_log                    with p-value 1.71728e-13
Add  Neighborhood_NridgHt           with p-value 7.05633e-12
Add  BldgType_Duplex                with p-value 4.30647e-11
Add  Neighborhood_NAmes             with p-value 2.25803e-09
Add  Neighborhood_SWISU             with p-value 5.40743e-09
Add  Neighborhood_BrkSide           with p-value 8.79638e-10
Add  Neighborhood_Sawyer            with p-value 6.92011e-09
Add  Neighborhood_NoRidge           with p-value 5.87105e-08
Add  Neighborhood_Somer

### Build the final model again in Statsmodels

In [ ]:
# Your code here

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [ ]:
# Your code here

Fit the linear regression model again using the 5 selected columns

In [ ]:
# Your code here

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [ ]:
# Your code here

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [ ]:
# Your code here

# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 